In [1]:
import os
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy

import dataset
import test_lstm

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

2023-03-16 18:40:41.082083: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_path = ['finalized_code/data/OpenFace/trial/lie/', 'finalized_code/data/OpenFace/MU3D/lie/', 'finalized_code/data/OpenFace/BOL/lie/'] #60 entries
truth_path = ['finalized_code/data/OpenFace/trial/truth/', 'finalized_code/data/OpenFace/MU3D/lie/', 'finalized_code/data/OpenFace/BOL/truth/'] #61 entries

In [3]:
# # no split by person
X, Y = dataset.preprocessing(truth_path, lie_path, numOfFrames=20)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

Processing finalized_code/data/OpenFace/trial/truth/
Processing finalized_code/data/OpenFace/MU3D/lie/
Processing finalized_code/data/OpenFace/BOL/truth/
Processing finalized_code/data/OpenFace/trial/lie/
Processing finalized_code/data/OpenFace/MU3D/lie/
Processing finalized_code/data/OpenFace/BOL/lie/


In [4]:
# split by person
# xTrain, yTrain, xTest, yTest = dataset.createDatasetLSTM(lie_trial_path, truth_trial_path, 0.2, byPerson=False)

In [5]:
LSTM_NEURONS = 32
DROPOUT = 0
EPOCHS = 10
OPTIM = 'adam'
# LOSS = 'binary_crossentropy'
LOSS = BinaryCrossentropy(from_logits=False)

n_inputRows = xTrain.shape[0] 
n_timesteps = xTrain.shape[1]
n_features = xTrain.shape[2]

def LSTM_Model(neuro, drop, loss, optim, step, feat):
  model = Sequential()
  model.add(LSTM(neuro, return_sequences=False, batch_input_shape=(None, step, feat)))
  
  model.add(Dropout(drop))
  model.add(Dense(1, activation='sigmoid')) #softmax

  model.compile(loss=loss, optimizer=optim, metrics=['accuracy'])
  return model

MODEL = LSTM_Model(LSTM_NEURONS, DROPOUT, LOSS, OPTIM, n_timesteps, n_features)
MODEL.summary()
MODEL.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=EPOCHS, verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5504      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 5,537
Trainable params: 5,537
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
11397/11397 [==============================] - 72s 6ms/step - loss: 0.6681 - accuracy: 0.5490 - val_loss: 0.6584 - val_accuracy: 0.5629
Epoch 2/10
11397/11397 [==============================] - 63s 6ms/step - loss: 0.6541 - accuracy: 0.5661 - val_loss: 0.6500 - val_accuracy: 0.5716
Epoch 3/10
11397/11397 [

In [8]:
prediction = MODEL.predict(xTest[0:250])
label = yTest[0:250]

for i in range(len(prediction)):
  if prediction[i] > 0.5:
    prediction[i] = 1
  else:
    prediction[i] = 0

accuracy = 0
for i in range(len(label)):
  if prediction[i] == label[i]:
    accuracy += 1
  
print("Accuracy: ", accuracy/len(label))

8/8 [==============================] - 0s 4ms/step
Accuracy:  0.584


In [14]:
import importlib
importlib.reload(test_lstm)

lie_test_path = './test/Deceptive/processed/'
label = 1

prediction_test = test_lstm.perdictSingleVideo(lie_test_path, MODEL, label, numOfFrames=10)

11/11 [==============================] - 0s 2ms/step
test.csv  accuracy:  0.008595988538681949  label:  1
39/39 [==============================] - 0s 2ms/step
test4.csv  accuracy:  0.6378334680679062  label:  1
35/35 [==============================] - 0s 2ms/step
test5.csv  accuracy:  0.6294602012808783  label:  1
34/34 [==============================] - 0s 2ms/step
test2.csv  accuracy:  0.5864661654135338  label:  1
23/23 [==============================] - 0s 3ms/step
test3.csv  accuracy:  0.5334242837653479  label:  1
17/17 [==============================] - 0s 2ms/step
test1.csv  accuracy:  0.851782363977486  label:  1
